In [2]:
import math
import numpy as np
import open3d as o3d
from sklearn.cluster import MeanShift
import trimesh
from utils import trimesh_to_o3d, o3d_to_trimesh, smooth_normals, calc_local_sampling_spacing, find_cyl_neighbours

In [3]:
def trilateral_shift(pcd, scan_id, D, alpha=1.0, sigma_factor=1.0):
    pts = np.asarray(pcd.points);  nrm = np.asarray(pcd.normals)
    new_pts = pts.copy()
    for i, (p, n, d) in enumerate(zip(pts, nrm, D)):
        r = alpha * d;  h = 2*alpha*d
        cyl_mask = np.logical_and.reduce([
            np.linalg.norm(np.cross(pts - p, n), axis=1) < r,   # radial
            np.abs((pts - p) @ n) < h/2,                        # height
            scan_id != scan_id[i]                               # other scan
        ])
        idx = np.where(cyl_mask)[0]
        if idx.size == 0: continue
        P = pts[idx]; N = nrm[idx]
        ri = np.linalg.norm(P - p - ((P - p) @ n)[:,None]*n, axis=1)
        hi = (P - p) @ n
        rho = 1./(D[idx]**2); rho_max = rho.max()
        w = np.exp(-ri**2/(2*r**2*sigma_factor**2)) \
            * np.exp(-hi**2/(2*h**2*sigma_factor**2)) \
            * np.exp(-(rho-rho_max)**2/(2*(rho_max-rho.min())**2*sigma_factor**2))
        shift = (w*hi).sum() / (w.sum() + 1e-12)
        new_pts[i] = p + shift * n
    pcd.points = o3d.utility.Vector3dVector(new_pts)
    return pcd

In [4]:
mesh1 = trimesh.load_mesh('meshes/bottle_1.ply')
mesh2 = trimesh.load_mesh('meshes/bottle_2.ply')

mesh1_o3d = trimesh_to_o3d(mesh1)
mesh2_o3d = trimesh_to_o3d(mesh2)

In [5]:
points1 = np.asarray(mesh1.vertices)
points2 = np.asarray(mesh2.vertices)
pointclouds = (points1, points2)
points = np.vstack(pointclouds)

tree = o3d.geometry.KDTreeFlann(points.T)

mesh1_o3d.compute_vertex_normals()
mesh2_o3d.compute_vertex_normals()

normals1 = np.asarray(mesh1_o3d.vertex_normals)
normals2 = np.asarray(mesh2_o3d.vertex_normals)
normals = np.concat([normals1, normals2], axis=0)

ids = np.concat([np.ones(len(these_points)) * i for (i, these_points) in enumerate(pointclouds)])

In [6]:
normals = smooth_normals(points, normals, k=8, T=0.7, n_iters=5)

In [7]:
lss1, lsd1 = calc_local_sampling_spacing(mesh1.vertices, np.asarray(mesh1_o3d.vertices))
lss2, lsd2 = calc_local_sampling_spacing(mesh2.vertices, np.asarray(mesh2_o3d.vertices))
lsss = (lss1, lss2)
local_sampling_spacing = np.concat(lsss)

global_avg_spacing = (1/len(lsss)) * np.sum([(1/len(lss)) * np.sum(lss) for lss in lsss])

In [8]:
j = 0

pj = points[j]
nj = normals[j]

alph = 2.0
Dpj = local_sampling_spacing[j]
r_c = alph * Dpj
h_c = 2*alph * Dpj
R = np.sqrt(r_c**2 + (h_c/2)**2)

N, idx, d2 = tree.search_radius_vector_3d(pj, R)
assert N > 1, 'need to find more than one point'

neighbour_idx = []
for k in np.asarray(idx[1:]):
    pk = points[k]
    h = abs(np.dot(nj, pk - pj))
    
    d = pk - pj
    dist2 = np.dot(d, d)
    r2 = max(dist2 - h*h, 0.0)
    r = math.sqrt(r2)
    
    if h <= 0.5*h_c and r <= r_c:
        neighbour_idx.append(k)

print(np.array([int(n) for n in neighbour_idx]))

[ 4  5  3 12  1 13 11 14  6 10 25 24 23 26]


In [10]:
j = 0
pj = points[j]
nj = normals[j]
alph = 2.0
Dpj = local_sampling_spacing[j]
neighbour_idx = find_cyl_neighbours(pj, nj, Dpj, alph, points, tree)
print(neighbour_idx)

[ 4  5  3 12  1 13 11 14  6 10 25 24 23 26]
